# PoC: History-based Recommendation

## Goal
Prototype a recommendation system that takes a **list of recently watched animes** (User History) and suggests similar content. 
This moves beyond the simple "Item-to-Item" recommendation to a "User-to-Item" approach.


## Assumption: Franchise Awareness
> **Assumption**: Users are generally aware of the franchises they are already watching. 
> If a user has *Naruto* in their history, recommending *Naruto Shippuden* is trivial and less valuable than recommending a *new* series like *Hunter x Hunter*. 
> **Refinement**: We explicitly filter out recommendations that contain the title of any anime in the history (e.g., exclude "One Piece: Movie 1" if "One Piece" is in history).

## Methodologies

We will compare two distinct strategies to handle the user's history:

### Strategy 1: The Centroid Method (Mean Vector)
**The Average Taste**

- **Concept**: We assume a user can be represented by the *average* of what they watch. If you watch a *Horror* anime and a *Comedy* anime, your "user vector" sits somewhere in the middle (e.g., *Dark Comedy*).
- **Math**:
  1. Retrieve the TF-IDF feature vector $\vec{v}_i$ for each anime $i$ in the history.
  2. Compute the User Vector: $\vec{u} = \frac{1}{N} \sum_{i=1}^{N} \vec{v}_i$
  3. Find the $k$ nearest neighbors to $\vec{u}$ in the entire dataset.
- **Pros**: Fast (1 query), holistic.
- **Cons**: Can dilute specific tastes. The average of "Naruto" and "Your Lie in April" might be a generic show that fits neither perfectly.

### Strategy 2: Multi-Query Aggregation
**The Eclectic Taste**

- **Definition**: "Eclectic" refers to a taste derived from a wide and diverse range of sources. This strategy assumes the user has multiple distinct interests rather than one single "average" preference.

- **Concept**: We respect that a user might have distinct, unrelated interests. We find recommendations for *each* show in history separately, then combine them.
- **Math**:
  1. For each anime $i$ in history, find its top $k$ neighbors $R_i = \{r_{i1}, r_{i2}, ...\}$.
  2. Pool all results $R_{total} = R_1 \cup R_2 \cup ...$
  3. Rank candidates by **Frequency** (how many history items voted for it?) and **Score**.
- **Pros**: Preserves distinct genres. Excellent for users with diverse taste.
- **Cons**: Slower ($N$ queries).


In [28]:
import sys
import os
from pathlib import Path

import pandas as pd
import numpy as np
from scipy.sparse import vstack

# Add project root to path to verify accessing src modules
current_dir = Path(os.getcwd()).resolve()
project_root = current_dir.parent
sys.path.append(str(project_root))

from src.pipeline.inference import load_models, load_processed_data

## Setup & Data Loading

In [29]:
knn, vectorizer = load_models()
df = load_processed_data()

2026-02-04 11:51:34,092 - inference - INFO - Loading models...


2026-02-04 11:51:34,278 - inference - INFO - Loading data from C:\Users\sayye\OneDrive\Documents\GitHub\AniMate\artifacts\vector_embeddings.pkl...


In [30]:
df.head(1)

,mal_id,title,english title,japanese title,episodes,release year,status,air date,genres,themes,...,favorites,content rating,source,duration,url,image url,score,stemmed_synopsis,producer,combined_features
0,16498,Shingeki no Kyojin,Attack on Titan,進撃の巨人,25.0,2013.0,Finished Airing,"Apr 7, 2013 to Sep 29, 2013","Action, Award Winning, Drama, Suspense","Gore, Military, Survival",...,186403,R - 17+ (violence & profanity),Manga,24 min per ep,https://myanimelist.net/anime/16498/Shingeki_n...,https://cdn.myanimelist.net/images/anime/10/47...,8.57,centuri ago mankind slaughter near extinct mon...,NaN,Shingeki no Kyojin Attack on Titan 進撃の巨人 Actio...


## Helper Functions

We need a robust way to find the vector for a given anime title. 
Since our model works on `combined_features`, we must:
1. Find the dataframe row for the title.
2. Extract the `combined_features` text.
3. Transform it using the loaded `vectorizer`.

In [31]:
def get_anime_vector(title_query, dataframe, vectorizer):
    """
    Finds an anime by title and returns its TF-IDF vector.
    Returns (None, None) if not found.
    """
    # Case-insensitive search
    match = dataframe[dataframe['title'].str.contains(title_query, case=False, na=False, regex=False)]
    
    if match.empty:
        # Try English title
        match = dataframe[dataframe['english title'].str.contains(title_query, case=False, na=False, regex=False)]
        
    if match.empty:
        print(f"Warning: '{title_query}' not found in database.")
        return None, None
    
    # Take the first match
    row = match.iloc[0]
    text_feature = row['combined_features']
    
    # Vectorize
    vector = vectorizer.transform([text_feature])
    return vector, row['title']

In [32]:
def get_user_history_vectors(history_titles, df, vectorizer):
    """
    Converts a list of titles into a matrix of vectors.
    """
    vectors = []
    found_titles = []
    
    for title in history_titles:
        vec, true_title = get_anime_vector(title, df, vectorizer)
        if vec is not None:
            vectors.append(vec)
            found_titles.append(true_title)
            
    if not vectors:
        return None, []
        
    return vstack(vectors), found_titles

## Implementation: Strategy 1 (Centroid)
 
We calculate $\vec{u} = \text{mean}(\text{vectors})$. Then query the KNN model once.

In [33]:
def recommend_centroid(history_titles, df, vectorizer, knn, top_k=5):
    vectors, found_titles = get_user_history_vectors(history_titles, df, vectorizer)
    
    if vectors is None:
        return pd.DataFrame() # No history found
    
    # Calculate Centroid (Mean Vector)
    # Note: TF-IDF vectors are sparse, but mean works fine mathematically
    user_vector = vectors.mean(axis=0)

    # Fix for sklearn: Convert np.matrix to np.array
    import numpy as np
    user_vector = np.asarray(user_vector)
    
    # Search KNN
    # We ask for more than k to allow filtering history items
    distances, indices = knn.kneighbors(user_vector, n_neighbors=50) # Request 50 to allow for franchise filtering
    
    # Process Results
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        row = df.iloc[idx]
        title = row['title']
        
        # Filter out history
        # FRANCHISE FILTERING ASSUMPTION:
        # We assume the user is aware of the franchise they are watching.
        # If they watched 'Naruto', we exclude 'Naruto Shippuden' to encourage discovery of NEW series.
        is_franchise_duplicate = any(history_item.lower() in title.lower() for history_item in found_titles)
        if title in found_titles or is_franchise_duplicate:
            continue
            
        results.append({
            'title': title,
            'genres': row.get('genres'),
            'themes': row.get('themes'),
            'distance': dist,
            'strategy': 'Centroid'
        })
        
        if len(results) >= top_k:
            break
            
    return pd.DataFrame(results)

## Implementation: Strategy 2 (Multi-Query)

We query for *each* history item, then aggregate. 
Scoring rule: 
- 1 point for being a neighbor.
- Boost by (1 - distance) to value closer matches.

In [34]:
def recommend_multiquery(history_titles, df, vectorizer, knn, top_k=5):
    vectors, found_titles = get_user_history_vectors(history_titles, df, vectorizer)
    
    if vectors is None:
        return pd.DataFrame()
    
    candidates = {}
    
    # Iterate through each history item
    # vectors is a sparse matrix, iterate rows
    for i in range(vectors.shape[0]):
        vec = vectors.getrow(i)
        
        # Get neighbors for this specific item
        dists, idxs = knn.kneighbors(vec, n_neighbors=20) # Request 20 to allow for franchise filtering
        
        for d, idx in zip(dists[0], idxs[0]):
            row = df.iloc[idx]
            title = row['title']
            
            # FRANCHISE FILTERING ASSUMPTION:
            # We assume the user is aware of the franchise they are watching.
            # If they watched 'Naruto', we exclude 'Naruto Shippuden' to encourage discovery of NEW series.
            is_franchise_duplicate = any(history_item.lower() in title.lower() for history_item in found_titles)
            if title in found_titles or is_franchise_duplicate:
                continue
                
            # Scoring Logic
            # Score = Sum of (1 - distance) scores from all triggers
            # A perfect match (dist=0) gives 1.0
            score_boost = 1.0 - d
            
            if title not in candidates:
                candidates[title] = {
                    'row': row, 
                    'total_score': 0, 
                    'frequency': 0,
                    'min_distance': 1.0
                }
            
            candidates[title]['total_score'] += score_boost
            candidates[title]['frequency'] += 1
            candidates[title]['min_distance'] = min(candidates[title]['min_distance'], d)
            
    # Convert to DataFrame
    results = []
    for title, data in candidates.items():
        results.append({
            'title': title,
            'genres': data['row'].get('genres'),
            'themes': data['row'].get('themes'),
            'score': data['total_score'],
            'frequency': data['frequency'],
            'best_dist': data['min_distance'],
            'strategy': 'Multi-Query'
        })
        
    # Sort by Score desc
    results_df = pd.DataFrame(results)
    if not results_df.empty:
        results_df = results_df.sort_values('score', ascending=False).head(top_k)
        
    return results_df

## Comparison Run

In [37]:
# TEST CASE 1: The "Action/Shounen" Fan
# Logic: Should recommend other high-stakes action shows
history_1 = ["Naruto", "One Piece", "Bleach"]

print(f"--- Recommendations for History: {history_1} ---")
print("\n[Strategy 1: Centroid]")
res_c = recommend_centroid(history_1, df, vectorizer, knn)
if not res_c.empty:
    display(res_c[['title', 'distance', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")

print("\n[Strategy 2: Multi-Query]")
res_mq = recommend_multiquery(history_1, df, vectorizer, knn)
if not res_mq.empty:
    display(res_mq[['title', 'score', 'frequency', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")


--- Recommendations for History: ['Naruto', 'One Piece', 'Bleach'] ---

[Strategy 1: Centroid]


,title,distance,genres,themes
0,Ichigo Mashimaro Encore,0.757044,Slice of Life,"CGDCT, Iyashikei"
1,Yume-iro Pâtissière SP Professional,0.760591,"Gourmet, Slice of Life",NaN



[Strategy 2: Multi-Query]


,title,score,frequency,genres,themes
0,Ichigo Mashimaro Encore,0.405695,1,Slice of Life,"CGDCT, Iyashikei"
1,Yume-iro Pâtissière SP Professional,0.368532,1,"Gourmet, Slice of Life",NaN
2,Tokyo Mew Mew,0.304171,1,"Romance, Sci-Fi",Mahou Shoujo
3,Tokyo Mew Mew New ♡,0.291607,1,"Romance, Sci-Fi",Mahou Shoujo
4,Yume-iro Pâtissière,0.255501,1,Gourmet,School


In [38]:
# TEST CASE 2: The "Mixed Taste" User
# Logic: Likes Psychological Thrillers AND Comedy. 
# Centroid might struggle. Multi-Query should find gems for both.
history_2 = ["Death Note", "One Punch Man"]

print(f"--- Recommendations for History: {history_2} ---")
print("\n[Strategy 1: Centroid]")
res_c = recommend_centroid(history_2, df, vectorizer, knn)
if not res_c.empty:
    display(res_c[['title', 'distance', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")

print("\n[Strategy 2: Multi-Query]")
res_mq = recommend_multiquery(history_2, df, vectorizer, knn)
if not res_mq.empty:
    display(res_mq[['title', 'score', 'frequency', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")


--- Recommendations for History: ['Death Note', 'One Punch Man'] ---

[Strategy 1: Centroid]


,title,distance,genres,themes
0,Shinigami no Ballad.,0.837587,"Drama, Supernatural",Psychological
1,Death Parade,0.838662,"Drama, Fantasy, Suspense","Adult Cast, High Stakes Game, Psychological"
2,Boku no Hero Academia: Hero Note,0.841307,Action,"School, Super Power"
3,One Outs,0.841324,"Sports, Suspense","Adult Cast, Psychological, Team Sports"
4,Übel Blatt,0.844271,"Action, Adventure, Fantasy",NaN



[Strategy 2: Multi-Query]


,title,score,frequency,genres,themes
0,Shinigami no Ballad.,0.233253,1,"Drama, Supernatural",Psychological
1,Death Parade,0.204931,1,"Drama, Fantasy, Suspense","Adult Cast, High Stakes Game, Psychological"
2,Tesla Note,0.193137,1,"Action, Suspense",NaN
18,Sweat Punch,0.188596,1,Fantasy,"Historical, Mecha"
19,One Outs,0.174484,1,"Sports, Suspense","Adult Cast, Psychological, Team Sports"


In [ ]:
# TEST CASE 3: Single Anime Input
# Logic: Should recommend other high-stakes action shows
history_1 = ["Mashle"]

print(f"--- Recommendations for History: {history_1} ---")
print("\n[Strategy 1: Centroid]")
res_c = recommend_centroid(history_1, df, vectorizer, knn)
if not res_c.empty:
    display(res_c[['title', 'distance', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")

print("\n[Strategy 2: Multi-Query]")
res_mq = recommend_multiquery(history_1, df, vectorizer, knn)
if not res_mq.empty:
    display(res_mq[['title', 'score', 'frequency', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")


--- Recommendations for History: ['Mashle'] ---

[Strategy 1: Centroid]


,title,distance,genres,themes
0,Magic Maker: Isekai Mahou no Tsukurikata,0.714008,Fantasy,"Isekai, Reincarnation"
1,Nisekoi: OVA,0.725768,"Comedy, Romance","Harem, Parody"
2,Wizard Barristers: Benmashi Cecil,0.746961,Fantasy,"Mecha, Urban Fantasy"
3,Kabushikigaisha Magi-Lumière,0.750540,Action,"Adult Cast, Mahou Shoujo, Workplace"
4,Magic-Kyun! Renaissance,0.754043,"Fantasy, Romance","Music, Reverse Harem, School, Visual Arts"



[Strategy 2: Multi-Query]


,title,score,frequency,genres,themes
0,Magic Maker: Isekai Mahou no Tsukurikata,0.285992,1,Fantasy,"Isekai, Reincarnation"
1,Nisekoi: OVA,0.274232,1,"Comedy, Romance","Harem, Parody"
2,Wizard Barristers: Benmashi Cecil,0.253039,1,Fantasy,"Mecha, Urban Fantasy"
3,Kabushikigaisha Magi-Lumière,0.249460,1,Action,"Adult Cast, Mahou Shoujo, Workplace"
4,Magic-Kyun! Renaissance,0.245957,1,"Fantasy, Romance","Music, Reverse Harem, School, Visual Arts"


In [ ]:
# TEST CASE 3: Multiple differering choices
# Logic: Should add a lot of big names to see if the system can handle it
history_1 = ["Mashle", "Solo Leveling", "Demon Slayer", "Jujutsu Kaisen", "One Punch Man", "One Piece", "Naruto", "Bleach"]

print(f"--- Recommendations for History: {history_1} ---")
print("\n[Strategy 1: Centroid]")
res_c = recommend_centroid(history_1, df, vectorizer, knn)
if not res_c.empty:
    display(res_c[['title', 'distance', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")

print("\n[Strategy 2: Multi-Query]")
res_mq = recommend_multiquery(history_1, df, vectorizer, knn)
if not res_mq.empty:
    display(res_mq[['title', 'score', 'frequency', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")


--- Recommendations for History: ['Mashle', 'Solo Leveling', 'Demon Slayer', 'Jujutsu Kaisen', 'One Punch Man', 'One Piece', 'Naruto', 'Bleach'] ---

[Strategy 1: Centroid]


,title,distance,genres,themes
0,Mahoujin Guruguru (2017),0.787634,"Adventure, Comedy, Fantasy",Parody
1,Kaiko sareta Ankoku Heishi (30-dai) no Slow na...,0.796532,Fantasy,NaN
2,Hunter x Hunter Movie 2: The Last Mission,0.807080,"Action, Adventure, Fantasy",NaN
3,Yuusha ni Narenakatta Ore wa Shibushibu Shuush...,0.810673,"Comedy, Fantasy, Romance, Ecchi",NaN
4,Gensoumaden Saiyuuki,0.817839,"Action, Adventure, Drama, Fantasy",NaN



[Strategy 2: Multi-Query]


,title,score,frequency,genres,themes
68,Ichigo Mashimaro Encore,0.405695,1,Slice of Life,"CGDCT, Iyashikei"
69,Yume-iro Pâtissière SP Professional,0.368532,1,"Gourmet, Slice of Life",NaN
17,Hunter x Hunter Movie 2: The Last Mission,0.352683,1,"Action, Adventure, Fantasy",NaN
18,Hunter x Hunter,0.350756,1,"Action, Adventure, Fantasy",NaN
19,Hunter x Hunter (2011),0.349946,1,"Action, Adventure, Fantasy",NaN


In [44]:
# TEST CASE 4: Single Anime Input
# Logic: Just providing a single anime title to the system
history_1 = ["Mashle", "Solo Leveling", "Demon Slayer", "Jujutsu Kaisen", "One Punch Man", "One Piece", "Naruto", "Bleach"]

print(f"--- Recommendations for History: {history_1} ---")
print("\n[Strategy 1: Centroid]")
res_c = recommend_centroid(history_1, df, vectorizer, knn)
if not res_c.empty:
    display(res_c[['title', 'distance', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")

print("\n[Strategy 2: Multi-Query]")
res_mq = recommend_multiquery(history_1, df, vectorizer, knn)
if not res_mq.empty:
    display(res_mq[['title', 'score', 'frequency', 'genres', 'themes']])
else:
    print("No recommendations found (all filtered by franchise logic).")


--- Recommendations for History: ['Mashle', 'Solo Leveling', 'Demon Slayer', 'Jujutsu Kaisen', 'One Punch Man', 'One Piece', 'Naruto', 'Bleach'] ---

[Strategy 1: Centroid]


,title,distance,genres,themes
0,Mahoujin Guruguru (2017),0.787634,"Adventure, Comedy, Fantasy",Parody
1,Kaiko sareta Ankoku Heishi (30-dai) no Slow na...,0.796532,Fantasy,NaN
2,Hunter x Hunter Movie 2: The Last Mission,0.807080,"Action, Adventure, Fantasy",NaN
3,Yuusha ni Narenakatta Ore wa Shibushibu Shuush...,0.810673,"Comedy, Fantasy, Romance, Ecchi",NaN
4,Gensoumaden Saiyuuki,0.817839,"Action, Adventure, Drama, Fantasy",NaN



[Strategy 2: Multi-Query]


,title,score,frequency,genres,themes
68,Ichigo Mashimaro Encore,0.405695,1,Slice of Life,"CGDCT, Iyashikei"
69,Yume-iro Pâtissière SP Professional,0.368532,1,"Gourmet, Slice of Life",NaN
17,Hunter x Hunter Movie 2: The Last Mission,0.352683,1,"Action, Adventure, Fantasy",NaN
18,Hunter x Hunter,0.350756,1,"Action, Adventure, Fantasy",NaN
19,Hunter x Hunter (2011),0.349946,1,"Action, Adventure, Fantasy",NaN
